#### 과제1 : ham / spam 메일 데이터(불용어 안 뺀 상태) 전처리 하기

In [ ]:
import re
import urllib.request
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
import pandas as pd
from konlpy.tag import Okt
okt=Okt()
from string import punctuation

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# LSTM 기반 분류기 만들기
# 행렬 계산 및 데이터 프레임 다루기
import numpy as np
import pandas as pd
# 시각화용
import matplotlib.pyplot as plt
# 웹에 있 데이터 다루기
import urllib.request

# 데이터 셋 나누기(학습, 확인)
from sklearn.model_selection import train_test_split

# 단어(문자)를 수치화 및 패딩으로 자리 맞추기
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/10.%20RNN%20Text%20Classification/dataset/spam.csv", filename="spam.csv")
data = pd.read_csv('spam.csv', encoding='latin1')
print('총 샘플의 수 :',len(data))

총 샘플의 수 : 5572


In [ ]:
data.head(3)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN


In [ ]:
data.isnull().sum()

v1               0
v2               0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64

In [ ]:
data.columns

del data['Unnamed: 2']
del data['Unnamed: 3']
del data['Unnamed: 4']

data

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
data['v1']=data['v1'].replace(['ham', 'spam'],[0,1])
data

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [ ]:
data.columns=['label', 'title']
data

,label,title
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [ ]:
data.info()
print()
data.describe()
print()
data.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   5572 non-null   int64 
 1   title   5572 non-null   object
dtypes: int64(1), object(1)
memory usage: 87.2+ KB




label  title                                                                                                                                                                                                        
0      Sorry, I'll call later                                                                                                                                                                                           30
       I cant pick the phone right now. Pls send a message                                                                                                                                                              12
       Ok...                                                                                                                                                                                                            10
1      Please call our customer service representative on FREEPHONE 0808 145 4742 between 9am-11pm as you have WON a guaranteed å£

In [ ]:
# 중복되는 제목 확인
print(data['title'].nunique())

# title이 중복인 데이터 지우기
data.drop_duplicates(subset=['title'],inplace=True)
print(len(data['title']))

5169
5169


In [ ]:
ydata=data['label']
xdata=data['title']

In [ ]:

# 실습 data는 y값의 편차가 심해, train으로 다 몰릴 수 있다.
# 층화추출로 해결 가능 stratify
# 층화추출: 종류에따라 균등하게 분리하여 추출하는 방법
# random_state= 난수
xtrain, xtest, ytrain, ytest=train_test_split(xdata,ydata,
                                              test_size=0.2,
                                              random_state=20230728,
                                              stratify=ydata)
print(len(ytest))
print(xtrain.iloc[0])
print(xtest.iloc[0])

1034
No. She's currently in scotland for that.
U still going to the mall?


In [ ]:
xtrain

1420            No. She's currently in scotland for that.
3801            No da. I am happy that we sit together na
1153    1000's of girls many local 2 u who r virgins 2...
3157    Thank u. IT BETTER WORK OUT CAUSE I WILL FEEL ...
4211    Missed call alert. These numbers called but le...
                              ...                        
698                        Mum ask Ì_ to buy food home...
5519    Can you pls send me that company name. In saib...
5503    Perhaps * is much easy give your account ident...
4689                                           Ok no prob
4456    Aight should I just plan to come up later toni...
Name: title, Length: 4135, dtype: object

In [ ]:
ytrain.value_counts()

0    3613
1     522
Name: label, dtype: int64

#####전처리 부분

In [ ]:
list(xtrain.values)

["No. She's currently in scotland for that.",
 'No da. I am happy that we sit together na',
 "1000's of girls many local 2 u who r virgins 2 this & r ready 2 4fil ur every sexual need. Can u 4fil theirs? text CUTE to 69911(å£1.50p. m)",
 'Thank u. IT BETTER WORK OUT CAUSE I WILL FEEL USED OTHERWISE',
 'Missed call alert. These numbers called but left no message. 07008009200',
 'What time do u get out?',
 'Arun can u transfr me d amt',
 "Oh, my love, it's soooo good to hear from you. Omg I missed you so much today. I'm sorry your having problems with the provider but thank you for tming me",
 'Send me your id and password',
 'Oh ok no prob..',
 "What's ur pin?",
 'To the wonderful Okors, have a great month. We cherish you guys and wish you well each day. MojiBiola',
 'FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GBP/mnth inc 3hrs 16 stop?txtStop',
 'We walked from my moms. Right on stagwood pass right on winterstone left 

In [ ]:
xtrain_list=[]
xtrain_list.extend(xtrain.values)
xtrain_list

["No. She's currently in scotland for that.",
 'No da. I am happy that we sit together na',
 "1000's of girls many local 2 u who r virgins 2 this & r ready 2 4fil ur every sexual need. Can u 4fil theirs? text CUTE to 69911(å£1.50p. m)",
 'Thank u. IT BETTER WORK OUT CAUSE I WILL FEEL USED OTHERWISE',
 'Missed call alert. These numbers called but left no message. 07008009200',
 'What time do u get out?',
 'Arun can u transfr me d amt',
 "Oh, my love, it's soooo good to hear from you. Omg I missed you so much today. I'm sorry your having problems with the provider but thank you for tming me",
 'Send me your id and password',
 'Oh ok no prob..',
 "What's ur pin?",
 'To the wonderful Okors, have a great month. We cherish you guys and wish you well each day. MojiBiola',
 'FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GBP/mnth inc 3hrs 16 stop?txtStop',
 'We walked from my moms. Right on stagwood pass right on winterstone left 

In [ ]:
# 결측치 없음
xtrain.isnull().sum()

0

In [ ]:
# 소문자 변환 진행
def pre_func(title):
    res=''.join(w.lower() for w in title if w not in punctuation)
    return res

pre_xtrain=[pre_func(x) for x in xtrain_list]
pre_xtrain[0:30]

['no shes currently in scotland for that',
 'no da i am happy that we sit together na',
 '1000s of girls many local 2 u who r virgins 2 this  r ready 2 4fil ur every sexual need can u 4fil theirs text cute to 69911å£150p m',
 'thank u it better work out cause i will feel used otherwise',
 'missed call alert these numbers called but left no message 07008009200',
 'what time do u get out',
 'arun can u transfr me d amt',
 'oh my love its soooo good to hear from you omg i missed you so much today im sorry your having problems with the provider but thank you for tming me',
 'send me your id and password',
 'oh ok no prob',
 'whats ur pin',
 'to the wonderful okors have a great month we cherish you guys and wish you well each day mojibiola',
 'freemsg txt call to no 86888  claim your reward of 3 hours talk time to use from your phone now subscribe6gbpmnth inc 3hrs 16 stoptxtstop',
 'we walked from my moms right on stagwood pass right on winterstone left on victors hill address is ltgt',
 'i

In [ ]:
# 불용어 제거 사전에 불용어 추가
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

post_xtrain = []

for sentence in pre_xtrain :
    post_xtrain.append(' '.join([word for word in sentence.split(' ') if word not in stopwords.words('english')]))

print(post_xtrain)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['shes currently scotland', 'da happy sit together na', '1000s girls many local 2 u r virgins 2  r ready 2 4fil ur every sexual need u 4fil text cute 69911å£150p', 'thank u better work cause feel used otherwise', 'missed call alert numbers called left message 07008009200', 'time u get', 'arun u transfr amt', 'oh love soooo good hear omg missed much today im sorry problems provider thank tming', 'send id password', 'oh ok prob', 'whats ur pin', 'wonderful okors great month cherish guys wish well day mojibiola', 'freemsg txt call 86888  claim reward 3 hours talk time use phone subscribe6gbpmnth inc 3hrs 16 stoptxtstop', 'walked moms right stagwood pass right winterstone left victors hill address ltgt', 'felt sonot conveying reason ese', 'anyways go gym whatever love smiles hope ok good day babe  miss much already', 'hi darlin youphone im athome youwanna chat', 'u attend ur driving lesson many times wk n day', 'freemsgfeelin kinda lnly hope u like 2 keep company jst got cam moby wanna c p

In [ ]:
# 객체 생성
tok=Tokenizer()
# xtrain을 corpus로 하여 fitting 및 단어별 인덱스 부여
tok.fit_on_texts(post_xtrain)
xtrain_enc=tok.texts_to_sequences(post_xtrain)

In [ ]:
xtrain_enc
w2i=tok.word_index
w2i

{'u': 1,
 'call': 2,
 '2': 3,
 'im': 4,
 'get': 5,
 'ur': 6,
 '4': 7,
 'go': 8,
 'dont': 9,
 'ok': 10,
 'ltgt': 11,
 'free': 12,
 'know': 13,
 'got': 14,
 'like': 15,
 'come': 16,
 'good': 17,
 'ill': 18,
 'want': 19,
 'time': 20,
 'day': 21,
 'love': 22,
 'one': 23,
 'text': 24,
 'send': 25,
 'need': 26,
 'going': 27,
 'home': 28,
 'still': 29,
 'see': 30,
 'r': 31,
 'txt': 32,
 'back': 33,
 'lor': 34,
 'reply': 35,
 'stop': 36,
 'think': 37,
 'sorry': 38,
 'tell': 39,
 'da': 40,
 'well': 41,
 'new': 42,
 'hi': 43,
 'take': 44,
 'n': 45,
 'today': 46,
 'please': 47,
 'phone': 48,
 'mobile': 49,
 'hey': 50,
 'night': 51,
 'ì': 52,
 'cant': 53,
 'oh': 54,
 'much': 55,
 'great': 56,
 'happy': 57,
 'later': 58,
 'claim': 59,
 'give': 60,
 'wat': 61,
 'week': 62,
 'make': 63,
 'hope': 64,
 'already': 65,
 'way': 66,
 'thats': 67,
 'pls': 68,
 'work': 69,
 'number': 70,
 'right': 71,
 'prize': 72,
 'yeah': 73,
 'tomorrow': 74,
 'yes': 75,
 'ask': 76,
 'dear': 77,
 'e': 78,
 'really': 79,
 '

In [ ]:
len(w2i)         #7831
xtrain.shape     # 4135
xtest.shape      # 1034
xtrain
type(xtrain_enc) # 2중 list 구조 중 가장 길이가 긴 list를 찾자.

# map(적용함수, 데이터)
# xtrain_enc에서 가장 긴 list의 길이
maxLen=max(map(len, xtrain_enc))
maxLen
# 어째서 0이?
# min(map(len, xtrain_enc))

# 응용 list의 평균 길이
# sum(map(len, xtrain_enc))/len(xtrain_enc)

66

In [ ]:
xtrain_padded=pad_sequences(xtrain_enc, maxlen=maxLen)

In [ ]:
xtrain_padded.shape

(4135, 66)

In [ ]:
# 단어번호 1번 부터 부여
# 0 패딩을 했으므로 1을 더하여 단어사전크기를 설정

vocab_size=len(w2i)+1

In [ ]:
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential

In [ ]:
model=Sequential()
model.add(Embedding(vocab_size, 32))
model.add(SimpleRNN(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history=model.fit(xtrain_padded, ytrain, epochs=5, batch_size=32,
                  validation_split=0.2)

Epoch 1/5
104/104 [==============================] - 28s 213ms/step - loss: 0.3270 - accuracy: 0.8748 - val_loss: 0.1902 - val_accuracy: 0.9311
Epoch 2/5
104/104 [==============================] - 11s 106ms/step - loss: 0.1237 - accuracy: 0.9592 - val_loss: 0.0758 - val_accuracy: 0.9770
Epoch 3/5
104/104 [==============================] - 10s 91ms/step - loss: 0.0524 - accuracy: 0.9846 - val_loss: 0.0779 - val_accuracy: 0.9758
Epoch 4/5
104/104 [==============================] - 10s 93ms/step - loss: 0.0406 - accuracy: 0.9897 - val_loss: 0.1342 - val_accuracy: 0.9589
Epoch 5/5
104/104 [==============================] - 9s 87ms/step - loss: 0.0226 - accuracy: 0.9933 - val_loss: 0.1563 - val_accuracy: 0.9456


In [ ]:
# from keras.models import load_model
# model.save('이름.확장자') : 모델 저장
# model = load_model('이름.확장자') : 모델 불러오기

In [ ]:
xtest_enc=tok.texts_to_sequences(xtest)
xtest_padded=pad_sequences(xtest_enc, maxlen=maxLen)

In [ ]:
# model.predict(xtest_padded)
model.evaluate(xtest_padded, ytest)

33/33 [==============================] - 0s 8ms/step - loss: 0.1269 - accuracy: 0.9603


[0.12691155076026917, 0.9603481888771057]